# notebook imagenette
Need to make pytorch transforms work.
To begin with crop image must be made image can in af databaunch can be scale to the same size

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import functools
from torch import torch
import torch.nn.functional as F

#let me be on my mac
def cuda(self, device=None, non_blocking=False) : return self
torch.Tensor.cuda = cuda

from lib.data.external import *
from lib.data.lists import *
from lib.learner.learner import *
from lib.learner.optimizers import *
from lib.image.image import *
from lib.model.model import*
from lib.model.modelmanager import*

from functools import *
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

## Imagenette data

In [ ]:

config   = IMAGENETTE_160_Configuration()
inputTfm = Transforms(Image2TensorImage(), NormalizeTransform(config.scale,config.mean,config.std))
files    = ImageList.from_files( untar_data(config.url) )
#ds       = ImageDataset(config, files, input_transform=inputTfm, output_transform=Numpy2Tensor(dtype=int)  )
ds       = ImageDataset(config, files, input_transform=inputTfm )
ds_train, ds_test = ds.split2train_test()

dl_train, dl_test = ds_train.dataloader(128,True), ds_test.dataloader(512, False)
databunch = DataBunch(dl_train, dl_test, config.channels_in, config.channels_out)

len(ds), len(ds_train), len(ds_test),len(dl_train), len(dl_test), 

(13394, 9469, 3925, 74, 8)

In [ ]:
ds_train[0]

(tensor([[[255., 250., 255.,  ..., 254., 254., 255.],
          [255., 242., 251.,  ..., 255., 255., 255.],
          [255., 252., 255.,  ..., 255., 255., 255.],
          ...,
          [249., 255., 255.,  ..., 254., 255., 254.],
          [255., 254., 255.,  ..., 254., 254., 255.],
          [255., 247., 254.,  ..., 253., 254., 255.]],
 
         [[255., 250., 255.,  ..., 254., 254., 255.],
          [255., 242., 251.,  ..., 255., 255., 255.],
          [255., 252., 255.,  ..., 255., 255., 255.],
          ...,
          [248., 255., 255.,  ..., 254., 255., 254.],
          [255., 253., 255.,  ..., 254., 254., 255.],
          [255., 246., 253.,  ..., 253., 254., 255.]],
 
         [[255., 250., 255.,  ..., 254., 254., 255.],
          [255., 242., 251.,  ..., 255., 255., 255.],
          [255., 252., 255.,  ..., 255., 255., 255.],
          ...,
          [246., 253., 253.,  ..., 254., 255., 254.],
          [253., 251., 253.,  ..., 254., 254., 255.],
          [253., 244., 251.,  .

In [ ]:
#layers_sizes = [32,64,128,256,512]
layers_sizes = [64,64,128,256]

sched = combine_scheds([0.3, 0.7], [sched_cos(0.3, 0.6), sched_cos(0.6, 0.2)]) 
cbfs_base = [TrainableModelCallback, TrainEvalCallback, OptimizerCallback, 
#        partial(ParamScheduler, 'lr', sched),
        partial(BatchTransformXCallback, view_tfm(3,64,64)),
        partial(MixUp,α=0.4),
        
        #CudaCallback,
        ProgressCallback,
       ]
cbfs = cbfs_base.copy() + [Recorder, partial(AvgStatsCallback,[accuracy])]
#cbfs_lr_Finder = cbfs_base.copy() + [LR_Finder]

In [ ]:
# average loss pr input sample at iteration 0
nb_classes = config.channels_out
initial_loss = -np.log( 1.0/nb_classes ) 
print(f"For equal distribution of cases pr class. nb_classes:{nb_classes}: \ninitial loss:    {initial_loss}\ninitial accuracy {1.0/nb_classes}")

For equal distribution of cases pr class. nb_classes:-1: 
initial loss:    nan
initial accuracy -1.0


<ipython-input-6-8b36cdff0019>:3: RuntimeWarning: invalid value encountered in log
  initial_loss = -np.log( 1.0/nb_classes )


## steppers=[sgd_step]

In [ ]:
layer = partial(conv_layer2, stride=2, bn=False, zero_bn=False, act=torch.nn.ReLU )
mm    = CnnModelManager( get_cnn_model(layers_sizes, databunch.c_in, databunch.c_out, layer ) )
mm.initialize(is_resnet=False)

learn = Learner( mm.model, databunch, loss_func=F.cross_entropy)
%time learn.fit(1, opt=SGD(sched,max_lr=0.1), cb_funcs=cbfs)

channels pr layers from input to output: [3, 64, 64, 128, 256, -1]
number of input and hidden layers: 4


RuntimeError: Trying to create tensor with negative dimension -1: [-1, 256]

In [ ]:
#learn.find_subcription_by_cls(LR_Finder).plot_loss(skip_end=15),plt.show()
learn.find_subcription_by_cls(Recorder).plot_loss(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()


## steppers=[weight_decay, sgd_step]

In [ ]:
layer = partial(conv_layer, stride=2, bn=False, zero_bn=False, act=nn.ReLU )
mm    = CnnModelManager( get_cnn_model(layers_sizes, databunch.c_in, databunch.c_out, layer ) )
mm.initialize(is_resnet=False)

learn = Learner( mm.model, databunch, loss_func=F.cross_entropy)
#print(opt.hypers[0]['lr']), print(opt.hypers[0]['wd'])
%time learn.fit(1, opt=SGD(sched,max_lr=0.1, max_wd=1e-4), cb_funcs=cbfs)

In [ ]:
learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_loss(),plt.show()

## momentum: steppers=[momentum_step,weight_decay]

In [ ]:
layer = partial(conv_layer, stride=2, bn=False, zero_bn=False, act=nn.ReLU )
mm    = CnnModelManager( get_cnn_model(layers_sizes, databunch.c_in, databunch.c_out, layer ) )
mm.initialize(is_resnet=False)

learn = Learner( mm.model, databunch, loss_func=F.cross_entropy)
%time learn.fit(1, opt=SGD_Momentum(sched,max_lr=0.08, moms=(0.85,0.95), max_wd=2e-4 ), cb_funcs=cbfs)

In [ ]:
#learn.find_subcription_by_cls(LR_Finder).plot_loss(skip_end=15),plt.show()
learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_loss(),plt.show()

## Adam: steppers=[adam_step]

In [ ]:
layer = partial(conv_layer, stride=2, bn=False, zero_bn=False, act=nn.ReLU )
mm    = CnnModelManager( get_cnn_model(layers_sizes, databunch.c_in, databunch.c_out, layer ) )
mm.initialize(is_resnet=False)
                        
learn = Learner( mm.model, databunch, loss_func=F.cross_entropy)
%time learn.fit(1, opt=Adam(sched,max_lr=3e-4, moms=(0.85,0.95), max_wd = 1e-6), cb_funcs=cbfs)

In [ ]:
#learn.find_subcription_by_cls(LR_Finder).plot_loss(skip_end=15),plt.show()
learn.find_subcription_by_cls(Recorder).plot_loss(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()

## LAMB: steppers=[lamb_step]

In [ ]:
layer = partial(conv_layer, stride=2, bn=False, zero_bn=False, act=nn.ReLU )
mm    = CnnModelManager( get_cnn_model(layers_sizes, databunch.c_in, databunch.c_out, layer ) )
mm.initialize(is_resnet=False)
                        
learn = Learner( mm.model, databunch, loss_func=F.cross_entropy)
%time learn.fit(3, opt=LAMB(sched,max_lr=3e-3, moms=(0.85,0.95), max_wd = 1e-6), cb_funcs=cbfs)

In [ ]:
#learn.find_subcription_by_cls(LR_Finder).plot_loss(skip_end=17),plt.show()
learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_loss(),plt.show()

## Adam with hooks

In [ ]:
layer = partial(conv_layer, stride=2, bn=False, zero_bn=False, act=nn.ReLU )
mm    = CnnModelManager( get_cnn_model(layers_sizes, databunch.c_in, databunch.c_out, layer ) )
mm.initialize(is_resnet=False)
                        
with Hooks(mm.model, append_stats) as hooks: 
    learn = Learner( mm.model, databunch, loss_func=F.cross_entropy)
    learn.fit(1, opt=Adam(sched,max_lr=2e-4, moms=(0.85,0.95), max_wd = 1e-4), cb_funcs=cbfs)
    plot_layer_stats( hooks )

## LAMB with hooks

In [ ]:
layer = partial(conv_layer, stride=2, bn=False, zero_bn=False, act=nn.ReLU )
mm    = CnnModelManager( get_cnn_model(layers_sizes, databunch.c_in, databunch.c_out, layer ) )
mm.initialize(is_resnet=False)

with Hooks(mm.model, append_stats) as hooks: 
    learn = Learner( mm.model, databunch, loss_func=F.cross_entropy)
    learn.fit(1, opt=LAMB(sched,max_lr=1e-2, moms=(0.85,0.95), max_wd = 1e-6), cb_funcs=cbfs )
    plot_layer_stats( hooks )